## Homework 8
### Section 2 LTCM Risk Decomposition
#### 2.1 Summary Stats

In [58]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
returns_data = pd.read_excel('../data/gmo_analysis_data.xlsx', sheet_name='returns (total)')
returns_data.rename(columns={'Unnamed: 0':'Date'},inplace=True)
returns_data = returns_data.set_index('Date')

risk_free_rate = pd.read_excel('../data/gmo_analysis_data.xlsx', sheet_name='risk-free rate')
risk_free_rate.rename(columns={'Unnamed: 0':'Date'},inplace=True)
risk_free_rate['Date'] = pd.to_datetime(risk_free_rate['Date'])
risk_free_rate = risk_free_rate.set_index('Date')
returns_data['RF'] = risk_free_rate
SPY_ex = pd.DataFrame(returns_data['SPY'] - returns_data['RF'], columns=['SPY'])
display(SPY_ex.head())


ltcm_data = pd.read_excel('../data/ltcm_exhibits_data.xlsx', sheet_name='Exhibit 2',  skiprows=3)
ltcm_data.drop(ltcm_data.tail(4).index,inplace=True)
ltcm_data.rename(columns={'Unnamed: 0':'Date 1', 'Unnamed: 1': 'Fund Capital', 'Unnamed: 2': 'Gross Returns', 'Unnamed: 3': 'Net Returns'},inplace=True)
ltcm_data = ltcm_data.drop(columns=['Date 1', 1])

ltcm_data.index = SPY_ex.loc['1994-03':'1998-07'].index
ltcm_data = ltcm_data.join(SPY_ex).join(risk_free_rate)
ltcm_data['Gross Returns (excess)'] = ltcm_data['Gross Returns'] - ltcm_data['US3M']
ltcm_data['Net Returns (excess)'] = ltcm_data['Net Returns'] - ltcm_data['US3M']

display(ltcm_data.head())

,SPY
Date,
1993-02-28,0.008159
1993-03-31,0.019949
1993-04-30,-0.028064
1993-05-31,0.024361
1993-06-30,0.001084


,Fund Capital,Gross Returns,Net Returns,SPY,US3M,Gross Returns (excess),Net Returns (excess)
Date,,,,,,,
1994-03-31,1.1,-0.011,-0.013,-0.044868,0.002967,-0.013967,-0.015967
1994-04-30,1.1,0.014,0.008,0.007904,0.003308,0.010692,0.004692
1994-05-31,1.2,0.068,0.053,0.012348,0.003592,0.064408,0.049408
1994-06-30,1.2,-0.039,-0.029,-0.026429,0.003550,-0.042550,-0.032550
1994-07-31,1.4,0.116,0.084,0.028668,0.003658,0.112342,0.080342


#### 2.1 a) Report mean, volatility and Sharpe of gross and net excess returns


In [63]:
def portfolio_stats_2(data):
    # Calculate the mean and annualize
    mean = data.mean() * 12

    # Volatility = standard deviation
    # Annualize the result with sqrt(12)
    vol = data.std() * np.sqrt(12)

    # Sharpe Ratio is mean / vol
    sharpe_ratio = mean / vol

    # Format for easy reading
    return round(pd.DataFrame(data = [mean, vol, sharpe_ratio], 
        index = ['Mean', 'Volatility', 'Sharpe']), 4)
    
print("Summary Stats")
portfolio_stats_2(ltcm_data[['Gross Returns (excess)', 'Net Returns (excess)', 'SPY']])

Summary Stats


,Gross Returns (excess),Net Returns (excess),SPY
Mean,0.2421,0.1554,0.1738
Volatility,0.1362,0.1118,0.1123
Sharpe,1.7769,1.3901,1.5479


#### 2.1 b) Report skewness, kurtosis, and (historic) VaR(0.05)

In [64]:
def tail_risk_stats(df, annual_fac=12):
    tr_df = df.skew().to_frame('Skew')
    tr_df['Kurtosis'] = df.kurtosis()
    tr_df['VaR (0.05)'] = df.quantile(0.05)
    return tr_df

print("Summary Stats Tails")
tail_risk_stats(ltcm_data[['Gross Returns (excess)', 'Net Returns (excess)', 'SPY']])

Summary Stats Tails


,Skew,Kurtosis,VaR (0.05)
Gross Returns (excess),-0.287725,1.586625,-0.030445
Net Returns (excess),-0.810239,2.926921,-0.026415
SPY,-0.433516,-0.362022,-0.046360


#### 2.1 c) Comment on how these stats compare to SPY and other assets we have seen. How much do they differ between gross and net?

We see in gross returns that the mean and Sharpe are higher than SPY, even though the vol is slightly higher. We see for net returns that the mean is slightly lower, while the vol stays basically the same, yielding a Sharpe ratio just slightly lower than SPY. In terms of Skew, net returns are much more skewed than SPY and gross returns. However, the VaR (0.05) of the SPY is the best, followed by gross returns and then net returns.

#### 2.2 a) Using net excess returns, calculate a regression and report alpha, beta, and R-squared.

In [67]:
def regress(y, X, intercept = True, annual_fac=12):
    if intercept == True:
        X_ = sm.add_constant(X)
        reg = sm.OLS(y, X_).fit()
        reg_df = reg.params.to_frame('Regression Parameters')
        reg_df.loc['R-squared'] = reg.rsquared
        reg_df.loc['const'] *= annual_fac
    else:
        reg = sm.OLS(y, X).fit()
        reg_df = reg.params.to_frame('Regression Parameters')
        reg_df.loc['R-squared'] = reg.rsquared
    
    return reg_df

regress(ltcm_data['Net Returns (excess)'], ltcm_data['SPY'])

,Regression Parameters
const,0.131527
SPY,0.137114
R-squared,0.018979


#### 2.2 b) From this regression, does LTCM appear to be a 'closet indexer'?
No. The R-squared value is very small, as is Beta

#### 2.2 c) From this regression, does LTCM appear to deliver excess returns beyond the risk premium we expect from market exposure?
Yes. We can see in the regression that alpha is quite high at ~0.13

#### 2.3 a) Check for non-linear market exposure. Run this regression and report B-1, B-2, and R-squared

In [69]:
ltcm_data['SPY (excess) squared'] = ltcm_data['SPY'] **2

regress(ltcm_data['Net Returns (excess)'], ltcm_data[['SPY','SPY (excess) squared']]) 

,Regression Parameters
const,0.155042
SPY,0.166878
SPY (excess) squared,-1.926746
R-squared,0.024321


#### 2.3 b) Does the quadratic market factor do much to increase the overall LTCM variation explained by the market?
While we see a small increase in R-Squared, overall I would say that no, adding in SPY squared does not increase the variation explained by the market.

#### 2.3 c) From the regression evidence, does LTCM's market exposure behave as if it is long market options or short market options?
It appears to be short market options, as seen by the negative Beta related to SPY squared.

#### 2.3 d) Should we describe LTCM as being positively or negatively exposed to market volatility?
Based on the negative Beta for SPY squared, I would say it is negatively exposed to market volatility.

#### 2.4 a) Try to pinpoint LTCM's nonlinear exposure. Run the regression and report Beta, Beta-up, Beta-down, and the R-squared statistic.

In [70]:
k1 = .03
k2 = -.03

ltcm_data['Up'] = (ltcm_data['SPY'] - k1).clip(0)
ltcm_data['Down'] = (k2 - ltcm_data['SPY']).clip(0)

regress(ltcm_data['Net Returns (excess)'], ltcm_data[['SPY','Up','Down']])

,Regression Parameters
const,0.101231
SPY,0.466610
Up,-0.782140
Down,1.289575
R-squared,0.055486


#### 2.4 b) Is LTCM long or short the call-like factor? The put-like factor?
LTCM is long the put-like factor (down) as seen by the positve Beta-Down, and short the call-like factor (up) as seen by the negative Beta-Up.

#### 2.4 c) Which factor moves LTCM more? The call-like factor or the put-like factor?
The put-like factor, as seen by the larger magnitutude it's beta has compared to the call-like factor.

#### 2.4 d) In the previous problem, you commented on whether LTCM is positively or negatively exposed to market volatility. Using this current regression, does this volatility exposure come more from being long the market’s upside? Short the market’s downside? Something else?
Based on this regression, the exposure must come more from being short the upside, because LTCM is short the call-like factor and long the put-like factor.

### Section 3 The FX Carry Trade

In [77]:
risk_free_rates = pd.read_excel('../data/fx_carry_data.xlsx', sheet_name='risk-free rates')
risk_free_rates = risk_free_rates.set_index('DATE')
# De-Annualized
risk_free_rates = risk_free_rates/12

log_risk_free_rates = np.log(1 + risk_free_rates)

display(log_risk_free_rates.head())

spot_fx_rates = pd.read_excel('../data/fx_carry_data.xlsx', sheet_name='fx rates')
spot_fx_rates = spot_fx_rates.set_index('DATE')
log_spot_fx_rates = np.log(spot_fx_rates)

display(log_spot_fx_rates.head())

,USD1M,GBP1M,EUR1M,CHF1M,JPY1M
DATE,,,,,
1999-01-31,0.000343,0.000411,0.000217,0.000083,0.000028
1999-02-28,0.000345,0.000385,0.000217,0.000086,0.000019
1999-03-31,0.000343,0.000369,0.000208,0.000083,0.000012
1999-04-30,0.000340,0.000367,0.000178,0.000067,0.000008
1999-05-31,0.000343,0.000368,0.000178,0.000069,0.000006


,USUK,USEU,USSZ,USJP
DATE,,,,
1999-01-31,0.498166,0.128481,-0.348401,-4.753590
1999-02-28,0.471690,0.094856,-0.371219,-4.776599
1999-03-31,0.478716,0.077702,-0.390351,-4.774322
1999-04-30,0.475302,0.054867,-0.422453,-4.782730
1999-05-31,0.471253,0.041334,-0.423960,-4.794798


#### 3.1 a,b,c) Report the mean, volatility, and Sharpe ratio. What differences do you see accross currencies?

In [80]:
# Take each currency that isn't USD and subtract the USD risk free rate
log_rf_ex = log_risk_free_rates.subtract(log_risk_free_rates['USD1M'], axis=0)[log_risk_free_rates.columns[1:]]

log_rf_ex_col = log_rf_ex.copy()

log_rf_ex_col.columns = log_spot_fx_rates.columns

# Now grab the log spot fx data and find the diff between t+1 and t, then add it to the diff between the risk free rates (log_rf_ex_col)
ret = (log_rf_ex_col.shift() + log_spot_fx_rates.diff()).dropna()

display(ret.head())
portfolio_stats_2(ret)

,USUK,USEU,USSZ,USJP
DATE,,,,
1999-02-28,-0.026408,-0.033752,-0.023078,-0.023324
1999-03-31,0.007067,-0.017282,-0.019391,0.001952
1999-04-30,-0.003387,-0.022970,-0.032362,-0.008739
1999-05-31,-0.004023,-0.013695,-0.001780,-0.012400
1999-06-30,-0.016021,-0.010970,-0.018111,-0.000833


,USUK,USEU,USSZ,USJP
Mean,-0.0077,0.0003,0.0179,-0.0008
Volatility,0.0863,0.0945,0.0985,0.0914
Sharpe,-0.0895,0.0029,0.1819,-0.0083


Across currencies, we see that returns are very small (even negative). We also see very small Sharpe ratios, with the highest being the Swiss franc with 0.1819.

#### 3.2 a) Do any of these stats contradict the (log version) of Uncovered Interest Parity (UIP)?
According to UIP, we should have mean excess returns of 0. USSZ is some evidence against UIP, with a mean return of 0.0179. The other currencies are very close to zero, and do not exactly contradict UIP.

#### 3.2 b) A long position in which currency offered the best Sharpe ratio over the sample?
Going long USSZ would have given the best mean return and Sharpe ratio.

#### 3.2 c) Are there any foreign currencies for which a long position earned a negative excess return over the sample?
Yes, both USUK and USJP returned a negative excess return, though it's so close to zero it's practically negligible.

#### 3.3 a) For each foreign currency, test whether interest rate differentials can predict growth in the foreign exchange rate. Make a table with columns corresponding to a different currency regression, report alpha, beta, and r-squared in the rows.

In [81]:
# let y be the t+1 - t rates of the log returns of spot fx
ys = log_spot_fx_rates.diff().dropna()

# let x be the negative of the previous subtraction (now is USD risk free rate - currency risk free rate)
Xs = -log_rf_ex.shift().dropna()

fx_reg_pred = pd.DataFrame(data = None, index = ['alpha-i','beta-i','R-squared'])

for i in range(0,len(ys.columns)):
    fx_reg_pred[ys.columns[i]] = regress(ys[ys.columns[i]],Xs[Xs.columns[i]])['Regression Parameters'].values
    
fx_reg_pred

,USUK,USEU,USSZ,USJP
alpha-i,-0.005854,0.007034,0.043550,-0.005973
beta-i,5.852952,-15.050115,-19.708839,4.435216
R-squared,0.000386,0.002611,0.003947,0.000498


#### 3.3 b) Suppose the foreign risk-free rate increases relative to the US rate.
#####   i) For which foreign currencies would we predict a relative strengthening of the USD in the following period?
We would expect a strengthening of the EU and SZ (the currencies with negative betas)
#####   ii) For which currencies would we predict relative weakening of the USD in the following period?
We would expect weakening on the UK and JP, due to their positive betas.
#####   iii) The FX predictability is strongest in the case of which foreign currency?
It is strongest in the case of USUK and USJP, which have the alphas closest to zero


#### 3.4 The Dynamic Carry Trade
#### 3.4 a)  Use your regression estimates from Problem 3 along with the formula above to calculate the fraction of months for which the estimated FX risk premium is positive. That is, for each i, calculate how often in the time-series we have expected value of r^i t+1 > 0

In [83]:
# The following code calculates the expected value of r^i t+1 (using equation given)
# De-annualize the alpha
alphas = (fx_reg_pred.loc['alpha-i'] / 12)
betas = fx_reg_pred.loc['beta-i']
Xs.columns = fx_reg_pred.columns

ret_ex_forecast = alphas.values + ((betas - 1) * Xs)

ret_ex_forecast.head()

,USUK,USEU,USSZ,USJP
DATE,,,,
1999-02-28,-0.000820,-0.001443,-0.001755,0.000585
1999-03-31,-0.000685,-0.001467,-0.001718,0.000619
1999-04-30,-0.000617,-0.001583,-0.001757,0.000639
1999-05-31,-0.000617,-0.002014,-0.002037,0.000643
1999-06-30,-0.000609,-0.002060,-0.002041,0.000660


In [85]:
# This calculates the percentage of time the risk premium is positive
month_frac = pd.DataFrame(data = None, columns = ret_ex_forecast.columns, index = ['% of Months'])

for col in ret_ex_forecast.columns:
    month_frac[col] = (len(ret_ex_forecast[ret_ex_forecast[col] > 0])/len(ret_ex_forecast)) * 100

display(month_frac)

,USUK,USEU,USSZ,USJP
% of Months,4.761905,53.113553,72.527473,33.699634


#### 3.4 b) Which currencies most consistently have a positive FX risk premium? And for which currencies does the FX risk premium most often go negative?
EU and SZ are both positive > 50% of the time, while UK and JP are are only positive 4% and 33% of the time respectively.

#### 3.4 c) Explain how we could use these conditional risk premia to improve the static carry trade returns calculated in Problem 1.
We can construct a portfolio trading strategy using the expected returns (like in HW 7) with weights that vary with the forecasts.